In [198]:
#Importamos librerías
import pandas as pd

In [172]:
#Archivo donde tomamos el set de datos correspondiente a contaminantes para su limpieza y normalización
#https://cdn.buenosaires.gob.ar/datosabiertos/datasets/agencia-de-proteccion-ambiental/calidad-aire/calidad-aire.csv
input_file_name = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/agencia-de-proteccion-ambiental/calidad-aire/calidad-aire.csv'

In [174]:
#Definimos el archivo donde guardar el set de datos luego de la limpieza y normalización
output_file_name = "contaminantes.xlsx"
output_sheet_name = "contaminantes"

In [175]:
#Carga del dataframe
df_contaminantes = pd.read_csv(input_file_name)

In [176]:
#Previsualización de los primeros registros
df_contaminantes.head()

,FECHA,HORA,CO_CENTENARIO,NO2_CENTENARIO,PM10_CENTENARIO,CO_CORDOBA,NO2_CORDOBA,PM10_CORDOBA,CO_LA_BOCA,NO2_LA_BOCA,PM10_LA_BOCA,CO_PALERMO,NO2_PALERMO,PM10_PALERMO
0,05MAR2010:00:00:00,19,s/d,s/d,NaN,s/d,31,NaN,s/d,s/d,NaN,0.91,NaN,NaN
1,05MAR2010:00:00:00,22,s/d,s/d,NaN,s/d,26,NaN,0.20,s/d,NaN,0.98,NaN,NaN
2,07MAR2010:00:00:00,9,s/d,s/d,NaN,s/d,28,NaN,0.35,s/d,NaN,1.23,NaN,NaN
3,08MAR2010:00:00:00,14,s/d,s/d,NaN,s/d,37,NaN,0.08,s/d,NaN,1.29,NaN,NaN
4,09MAR2010:00:00:00,18,s/d,s/d,NaN,s/d,40,NaN,0.20,24,NaN,1.00,NaN,NaN


In [177]:
#Cantidad de filas y columnas
df_contaminantes.shape

(104225, 14)

In [181]:
#Imputación del año según fecha de origen
df_contaminantes['Año'] = [ i[5:9] for i in df_contaminantes['FECHA'] ]

In [182]:
df_contaminantes = df_contaminantes.loc[ df_contaminantes.Año == '2020' ]

In [183]:
#Imputación del mes según fecha de origen
df_contaminantes['Mes'] = [ i[2:5] for i in df_contaminantes['FECHA'] ]

In [184]:
#Definición de los meses para su posterior normalización
MESES = { 'JAN' : '1', 'FEB' : '2', 'MAR' : '3', 'APR' : '4', 'MAY' : '5', 'JUN' : '6', 'JUL' : '7', 'AUG' : '8', 'SEP' : '9', 'OCT' : '10', 'NOV' : '11', 'DEC' : '12'}

In [185]:
#Normalización del mes
df_contaminantes["Mes"] = df_contaminantes["Mes"].map(MESES).astype(int)

In [186]:
#Imputación del tipo de registro
df_contaminantes['Tipo'] = 'Contaminantes'

In [187]:
#Filtrado de nulos seleccionando la estación Córdoba y Material Particulado PM10
df_contaminantes = df_contaminantes.dropna(subset=['PM10_CORDOBA'])

In [188]:
df_contaminantes = df_contaminantes.loc[ df_contaminantes.PM10_CORDOBA != 's/d' , :]

In [189]:
#Conversión de tipo a int seleccionando la estación Córdoba y Material Particulado PM10
df_contaminantes['PM10_CORDOBA'] = df_contaminantes['PM10_CORDOBA'].astype(int)

In [190]:
#Agrupación de datos seleccionando la estación Córdoba y Material Particulado PM10
df = df_contaminantes.groupby(['Año','Mes','Tipo'],as_index=False).mean()[['Año','Mes','Tipo','PM10_CORDOBA']].rename({'PM10_CORDOBA':'Total'},axis=1)
df['Total'] = round(df['Total'],2)
df

,Año,Mes,Tipo,Total
0,2020,1,Contaminantes,21.75
1,2020,2,Contaminantes,20.60
2,2020,3,Contaminantes,13.68
3,2020,4,Contaminantes,11.60
4,2020,5,Contaminantes,18.57
5,2020,6,Contaminantes,18.03
6,2020,7,Contaminantes,20.60
7,2020,8,Contaminantes,27.74
8,2020,9,Contaminantes,19.37
9,2020,10,Contaminantes,16.52


In [191]:
#Definición de intervalo según distribución de datos
df['Intervalo']=pd.cut(df['Total'],10,labels=[1,2,3,4,5,6,7,8,9,10])
df

,Año,Mes,Tipo,Total,Intervalo
0,2020,1,Contaminantes,21.75,7
1,2020,2,Contaminantes,20.60,6
2,2020,3,Contaminantes,13.68,2
3,2020,4,Contaminantes,11.60,1
4,2020,5,Contaminantes,18.57,5
5,2020,6,Contaminantes,18.03,4
6,2020,7,Contaminantes,20.60,6
7,2020,8,Contaminantes,27.74,10
8,2020,9,Contaminantes,19.37,5
9,2020,10,Contaminantes,16.52,4


In [192]:
#Definición relación de los datos entre diciembre y enero
enero = df.loc[ df.Mes == 1, ['Intervalo']].iloc[0,0]
diciembre = df.loc[ df.Mes == 12, ['Intervalo']].iloc[0,0]
diciembre_enero = int(((diciembre - enero)/enero)*100)
if diciembre_enero > 0:
    diciembre_enero = '+' + str(diciembre_enero) + ' %'
else:
    diciembre_enero = str(diciembre_enero) + ' %'

In [193]:
#Imputación relación de los datos entre diciembre y enero
df['Diciembre Vs Enero'] = diciembre_enero
df

,Año,Mes,Tipo,Total,Intervalo,Diciembre Vs Enero
0,2020,1,Contaminantes,21.75,7,-28 %
1,2020,2,Contaminantes,20.60,6,-28 %
2,2020,3,Contaminantes,13.68,2,-28 %
3,2020,4,Contaminantes,11.60,1,-28 %
4,2020,5,Contaminantes,18.57,5,-28 %
5,2020,6,Contaminantes,18.03,4,-28 %
6,2020,7,Contaminantes,20.60,6,-28 %
7,2020,8,Contaminantes,27.74,10,-28 %
8,2020,9,Contaminantes,19.37,5,-28 %
9,2020,10,Contaminantes,16.52,4,-28 %


In [194]:
#Resguardo de los datos para su posterior análisis 
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

df.to_excel(writer, index=False, sheet_name=output_sheet_name)

workbook  = writer.book
worksheet = writer.sheets[output_sheet_name]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')

worksheet.set_column('A:F', 30, format)

writer.save()